# Pretrained embeddings
The past couple of experiements have shown that training can be quite long and requires a very large dataset. Even the hundered of MB of current data is not enough. I note its not really stored in an efficent manner like HDF5 or something similar. 
One can save alot of time with task with small amounts of data by using pretrained models. 

In [1]:
import os

import numpy as np
path_to_glove_file = "D:/Documents/Code/AI_training/training_data/glove6b/glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file, encoding='UTF-8') as file:
    for line in file:
            word , coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs

print(f"number of word vectors = {len(embeddings_index)}")

number of word vectors = 400000


Slightly annoying, above for some reason ipython notebooks dont seem to have UTF-8 as the default encoding so you must specify this in the open function.

In [2]:
embedding_dim = 100
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vec = layers.TextVectorization(max_tokens=max_tokens, output_mode = "int", output_sequence_length=max_length)
from tensorflow import keras

train_ds = keras.utils.text_dataset_from_directory("D:/Documents/Code/AI_training/training_data/aclImdb_v1/aclImdb/train", batch_size=32)
val_ds = keras.utils.text_dataset_from_directory("D:/Documents/Code/AI_training/training_data/aclImdb_v1/aclImdb/val",batch_size=32)
test_ds = keras.utils.text_dataset_from_directory("D:/Documents/Code/AI_training/training_data/aclImdb_v1/aclImdb/test",batch_size=32)
text_only_train_ds = train_ds.map(lambda x,y:x)
text_vec.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)

vocab = text_vec.get_vocabulary()

word_index = dict(zip(vocab, range(len(vocab))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [3]:
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is None:
        embedding_matrix[i] = embedding_vector

You can see here we are using the models vectors in our own matrix. 

In [4]:
from tensorflow.keras import layers

embedding_layer = layers.Embedding(max_tokens, embedding_dim, embeddings_initializer = keras.initializers.Constant(embedding_matrix), trainable=False, mask_zero=True)

In [10]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model = keras.Model(inputs,outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________

In [11]:
callbacks = [keras.callbacks.ModelCheckpoint("glove_embed_sequence.keras", save_best_only=True)]
model.fit(int_train_ds.cache(), validation_data=int_val_ds.cache(), epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embed_sequence.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")


Epoch 1/10
100/625 [===>..........................] - ETA: 4:04 - loss: nan - accuracy: 0.5041

KeyboardInterrupt: 

Not certain why this doesnt train properly...